# 01. Azure AI Document Intelligence - Layout Model

> https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-4.0.0

## Create a AI Document Intelligence resource

To create a AI Document Intelligence resource in your Azure subscription:
Please follow the steps as specified https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-4.0.0


Get your newly created Document Intelligence service in the Azure portal and on the **Keys and Endpoint** page, copy the **Key1** and **Endpoint** values and paste them in the code cell below, replacing **YOUR_FORM_KEY** and **YOUR_FORM_ENDPOINT**.

Please ensure you have Python 3.10 version or above ie select **Python 3.10 - SDK v2** as kernal in AML. <br>
For VS Code you can select Python **Python 3.10** above or try to set up virtual envi by following steps https://code.visualstudio.com/docs/python/environments

In [8]:
# install azure-ai-formrecognizer python library and restart the kernal after installation
%pip install azure-ai-formrecognizer --upgrade --user

Note: you may need to restart the kernel to use updated packages.


## <mark>**A.Setting up AI Doc Intelligence endpoint and key**</mark>

In [2]:
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient, AnalyzeResult
from dotenv import load_dotenv

load_dotenv("credentials.env")
api_key=os.getenv("FORM_RECOGNIZER_KEY")
endpoint = os.getenv("FORM_RECOGNIZER_ENDPOINT") 

form_recognizer_client = DocumentAnalysisClient(endpoint, AzureKeyCredential(api_key))

In [3]:
print(endpoint)

https://eastus.api.cognitive.microsoft.com/


## B. Analysing the Layout Document

In [4]:
# sample document
layoutUrl = "https://azuremlworksho8836054660.blob.core.windows.net/bupa/bupa-insurance-limited-annual-report-and-accounts-2023.pdf"

# Start the document analysis using the Form Recognizer client
poller = form_recognizer_client.begin_analyze_document_from_url("prebuilt-layout",layoutUrl)
print(poller)

# Get the analysis result
result = poller.result()

# Extract document insights
# Check if the document contains handwritten content
if any([style.is_handwritten for style in result.styles]):
    print("Document contains handwritten content")
else:
    print("Document does not contain handwritten content")

# Iterate through each page in the document
for page in result.pages:
    print(f"----Analyzing layout from page #{page.page_number}----")
    print(
        f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}"
    )
    # Iterate through each line in the page
    for line_idx, line in enumerate(page.lines):
        words = line.get_words()
        #words = get_words(page, line)
        # Print information about each line
        print(
            f"...Line # {line_idx} has word count {len(words)} and text '{line.content}' "
            f"within bounding polygon '{line.polygon}'"
        )

        # Iterate through each word in the line and print its content and confidence
        for word in words:
            print(
                f"......Word '{word.content}' has a confidence of {word.confidence}"
            )
    
     # Iterate through each selection mark in the page
    for selection_mark in page.selection_marks:

        # Print information about each selection mark
        print(
            f"Selection mark is '{selection_mark.state}' within bounding polygon "
            f"'{selection_mark.polygon}' and has a confidence of {selection_mark.confidence}"
        )

# Iterate through each table in the document
for table_idx, table in enumerate(result.tables):
    # Print information about each table
    print(
        f"Table # {table_idx} has {table.row_count} rows and "
        f"{table.column_count} columns"
    )
    # Iterate through each region bounding the table
    for region in table.bounding_regions:
        # Print information about the location of the table on the page
        print(
            f"Table # {table_idx} location on page: {region.page_number} is {region.polygon}"
        )
    # Iterate through each cell in the table
    for cell in table.cells:
        # Print information about each cell
        print(
            f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'"
        )
        # Iterate through each region bounding the cell's content
        for region in cell.bounding_regions:
            # Print information about the region bounding the cell's content
            print(
                f"...content on page {region.page_number} is within bounding polygon '{region.polygon}'"
            )

Document contains handwritten content
----Analyzing layout from page #1----
Page has width: 8.2639 and height: 11.6806, measured with unit: inch
...Line # 0 has word count 3 and text 'Bupa Insurance Limited' within bounding polygon '[Point(x=3.1636, y=2.9252), Point(x=5.0952, y=2.9252), Point(x=5.0952, y=3.1077), Point(x=3.1636, y=3.1179)]'
......Word 'Bupa' has a confidence of 0.991
......Word 'Insurance' has a confidence of 0.991
......Word 'Limited' has a confidence of 0.993
...Line # 1 has word count 9 and text 'Annual Report and Accounts for the financial year ended' within bounding polygon '[Point(x=1.8505, y=3.5184), Point(x=6.3982, y=3.5184), Point(x=6.3982, y=3.7161), Point(x=1.8505, y=3.7161)]'
......Word 'Annual' has a confidence of 0.995
......Word 'Report' has a confidence of 0.995
......Word 'and' has a confidence of 0.996
......Word 'Accounts' has a confidence of 0.993
......Word 'for' has a confidence of 0.995
......Word 'the' has a confidence of 0.997
......Word 'finan

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## C. Extracted Layout Document insights/ response as a JSON format 

### **If we want to save it in JSON format**

In [5]:
import json
import datetime
import time
from azure.core.serialization import AzureJSONEncoder
from urllib.parse import urlparse

# Generate the unique file name
filename = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d%H%M%S') + "_" + os.path.splitext(os.path.basename(urlparse(layoutUrl).path))[0] + ".json"

# Parse and format the model response JSON 
analyze_result_dict = result.to_dict()

# Save the dictionary as JSON content in a JSON file, use the AzureJSONEncoder
# to help make types, such as dates, JSON serializable
with open(filename, 'w') as f:
    json.dump(analyze_result_dict, f, cls=AzureJSONEncoder, indent=4)

print("JSON response saved to file:", filename)


JSON response saved to file: 20240704234136_bupa-insurance-limited-annual-report-and-accounts-2023.json
